## 정보보호와 시스템보안
### Team Project #2

### 팀 정보
- 팀 이름 : Kim3
- 팀 번호 : 14
- 팀원 목록 : 김진명(20181599), 김태범(20181602), 김현우(20181603)

---

### 프로젝트1 TODO
1. Data Preprocessing
2. Data Scaling
3. IP Voting

### 프로젝트 데이터 셋
- 국민대학교 네트워크 플로우 데이터셋(5시간)   

### 참고자료
- 네트워크 플로우([Network Flow](https://checkmk.com/guides/network-flow-monitoring)):   
한번의 세션에서 출발지(Source IP)와 도착지(Destination IP) 사이에 오가는 패킷에 대한 통계적 데이터
### 주의사항
- 기존에 있는 파일과 폴더의 구조를 변경할 수 없음

### 제출
- `<팀 번호>_Project1.ipynb`파일을 팀장이 제출   
팀 번호 : 구글 스프레드 시트 행 인덱스

## 프로젝트 기본 설정

In [ ]:
from google.colab import drive

drive_path = '/content/drive'
drive.mount(drive_path)

Mounted at /content/drive


In [ ]:
# @kookminsecurityproject2022#
#!7z x /content/drive/MyDrive/2022_security_project1.zip
%cd "/content/drive/MyDrive/Security Project1"

/content/drive/MyDrive/Security Project1


In [ ]:
from Utils import open_pickle
import pandas as pd
import numpy as np

## train, valid 데이터 셋 불러오기
- 가상대학을 통해 다운받은 플로우 데이터를 불러온다.

In [ ]:
train_path = r"./train.csv"
valid_path = r"./project2_test.csv"

train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)

In [ ]:
# 학습, 검증 플로우 데이터 개수 확인
print(len(train_df), len(valid_df))

977270 325963


# 팀별 프로젝트 2

프로젝트 2는 프로젝트 1을 발전시켜 성능을 향상시키는 것이 목적입니다.

기존 코드 내용에 해당하는 전체 프로세스 (네트워크 플로우 -> 악성 외부 IP탐지)를 유지하면서,

악성 외부 IP탐지의 F-1 Score를 향상시키는 것입니다.

성능 향상을 위해 train, valid데이터를 원하는 방법으로 자유롭게 활용하시면 됩니다.

## 평가 기준 (F-1 Score 소수점 셋째자리 반올림 상대평가)

1.0 300점

0.99 290점

0.98 280점

0.97 270점

…

0.8 100점

0.8 미만 0점

성능 향상을 위한 가이드 및 기법에 대한 추가 QnA는 필요하다면 다음 주 중 진행될 예정입니다.(Ex. 피처 활용법 등)

## 데이터 셋

test데이터 셋은 train, valid와 동일한 양상이지만, IP가 다른 방식으로 익명화되어있기 때문에 train의 ip값 자체를 test단계에서 사용할 수 없습니다.(subnet구조는 기본적으로 유지) 즉, 다른 암호키 값으로 익명화된 IP입니다.

## 제출 방법

12월 2일(금) 18:00에 test데이터 셋 압축파일의 비밀번호가 공개됩니다.

train, valid 데이터 셋을 기반으로 test데이터 셋에서 악성 외부 IP를 탐지 후 제출하시면 됩니다.(12월 2일(금) 23:59까지)

외부 IP 셋은 제공됩니다.(탐지 대상)

해당 외부 IP셋에서 정상과 악성을 구분하고, 악성 IP만 제출하면 과제 종료입니다.

제출 형식은 csv 파일입니다.
컬럼은 외부IP, 예측 값으로 이루어져 있습니다.
예측 값은 악성은 1, 정상은 0으로 나타냅니다.
예시는 아래와 같습니다.

행0(컬럼 종류) : 'IP', 'Prediction'
행1 : <외부IP1>, 1
행2 : <외부IP2>, 0
행3 : <외부IP3>, 1
...

파일 이름 : <팀 번호>_Project2.csv

# 정답 파일 로드

In [ ]:
# 정상 외부 IP set
outer_benignIP_set = open_pickle("./outer_benign_IP.pkl")
# 악성 외부 IP set
outer_malIP_set = open_pickle("./outer_mal_IP.pkl")

In [ ]:
# Problem #3을 위해 dictionary형태로 변경
labelIP_dict = dict()

for ip in outer_benignIP_set:
    labelIP_dict[ip] = 0
for ip in outer_malIP_set:
    labelIP_dict[ip] = 1

In [ ]:
# 실기
outer_IP_set = open_pickle("./outer_ip_set.pkl")

labelIP_dict = dict()

for ip in outer_IP_set:
    labelIP_dict[ip] = 0

# 추가 Preprocessing 과정

## 전처리 데이터 프레임 생성

In [ ]:
tt_df = train_df.copy()
vv_df = valid_df.copy()
tt_df.columns

Index(['Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Flags',
       'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Std',
       'Packet Length Variance', 'Average Packet Size', 'Fwd Segment Size Avg',
       'Bwd Segment Size Avg', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max'

## Label PP

In [ ]:
from collections import Counter
from pprint import pprint as pp

In [ ]:
tt_df['Label'] = [int(x == 'Attack_IP') for x in train_df['Label']]
#vv_df['Label'] = [int(x == 'Attack_IP') for x in valid_df['Label']]
vv_df['Label'] = [0 for x in range(len(valid_df))]

In [ ]:
pp(Counter(tt_df['Label']))
pp(Counter(vv_df['Label']))

Counter({0: 571134, 1: 406136})
Counter({0: 325963})


## Protocol PP

In [ ]:
Counter(tt_df['Protocol'])

Counter({'TCP': 767561, '0': 87886, 'UDP': 121823})

In [ ]:
for col in ['UDP', 'TCP']:
    tt_df[col] = [int(p == col) for p in tt_df['Protocol']]
    vv_df[col] = [int(p == col) for p in vv_df['Protocol']]

In [ ]:
tt_df.head(3)

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,UDP,TCP
0,110.15.155.169,13850,97.2.159.1,53,TCP,......S.,03:52:00,3119967,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
1,110.15.155.169,43842,97.2.159.1,53,TCP,......S.,03:37:20,2991830,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
2,110.15.155.171,25918,97.2.159.1,53,TCP,......S.,03:10:28,3004658,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1


In [ ]:
# proto_idx = sorted(set(tt_df['Protocol']))
# proto_idx[1], proto_idx[2] = proto_idx[2], proto_idx[1]
# proto_idx

In [ ]:
# tt_df['Protocol'] = [proto_idx.index(p) for p in tt_df['Protocol']]
# vv_df['Protocol'] = [proto_idx.index(p) for p in vv_df['Protocol']]

In [ ]:
tt_df['Protocol'] = [int(p != '0') for p in tt_df['Protocol']]
vv_df['Protocol'] = [int(p != '0') for p in vv_df['Protocol']]

In [ ]:
tt_df['Protocol']

0         1
1         1
2         1
3         1
4         1
         ..
977265    1
977266    0
977267    1
977268    1
977269    1
Name: Protocol, Length: 977270, dtype: int64

## Flags PP

In [ ]:
pp(Counter(tt_df['Flags']).most_common(5))

[('......S.', 515406),
 ('........', 209709),
 ('...A.RS.', 145515),
 ('...A...F', 55356),
 ('CE.APRS.', 24564)]


In [ ]:
mf_cntr = Counter(tt_df[tt_df['Label'] == 1]['Flags'])
pp(mf_cntr.most_common(5))

[('......S.', 226926),
 ('...A.RS.', 120957),
 ('........', 44740),
 ('CE.APRS.', 7309),
 ('.....RS.', 1919)]


In [ ]:
bf_cntr = Counter(tt_df[tt_df['Label'] == 0]['Flags'])
pp(bf_cntr.most_common(5))

[('......S.', 288480),
 ('........', 164969),
 ('...A...F', 54479),
 ('...A.RS.', 24558),
 ('CE.APRS.', 17255)]


In [ ]:
# 공격 패킷에만 있는 flags
mf_cntr.keys() - bf_cntr.keys()

{'..U...S.', 'CE.A..SF', 'CE.A.RS.', 'CE.A.RSF', 'CE.AP.SF'}

In [ ]:
# 정상 패킷에만 있는 flags
bf_cntr.keys() - mf_cntr.keys()

{'.E.A..S.', 'CE.A..S.'}

In [ ]:
def flags_to_bool(flags: str, idx: int) -> int:
    return int(flags[idx] != '.')

In [ ]:
flag_columns = ['CWR', 'ECE', 'URG', 'ACK', 'PSH', 'RST', 'SYN', 'FIN']

for i, flag in enumerate(flag_columns):
    tt_df[flag] = tt_df['Flags'].map(lambda x: flags_to_bool(x, i))
    vv_df[flag] = vv_df['Flags'].map(lambda x: flags_to_bool(x, i))

In [ ]:
# def flags_to_bin(flags: str) -> int:
#     b = 0
#     for i in range(len(flags)):
#         if flags[i] != '.':
#             b += 2 << i
#     return b

In [ ]:
tt_df['Flags'] = tt_df['Flags'].map(lambda x: 8 - x.count('.'))
vv_df['Flags'] = vv_df['Flags'].map(lambda x: 8 - x.count('.'))

KeyboardInterrupt: ignored

In [ ]:
tt_df.head(3)

## Timestamp PP

In [ ]:
def timestamp_to_int(ts: str) -> int:
    hh, mm, ss = map(int, ts.split(':'))
    return 3600 * hh + 60 * mm + ss

In [ ]:
tt_df['Timestamp'] = tt_df['Timestamp'].map(timestamp_to_int)
vv_df['Timestamp'] = vv_df['Timestamp'].map(timestamp_to_int)

In [ ]:
tt_df.head(5)

## Src IP PP

In [ ]:
def is_outer_ip(ip: str) -> int:
    return int(ip in labelIP_dict)

In [ ]:
tt_df['Is Outer IP'] = tt_df['Src IP'].map(is_outer_ip)
vv_df['Is Outer IP'] = vv_df['Src IP'].map(is_outer_ip)

In [ ]:
Counter(tt_df['Is Outer IP'])

## Port PP

In [ ]:
src_tt = Counter(tt_df['Src Port'])
dst_tt = Counter(tt_df['Dst Port'])

src_vv = Counter(vv_df['Src Port'])
dst_vv = Counter(vv_df['Dst Port'])

In [ ]:
def port_to_prob_train_Src(port : int) -> float:
    return src_tt[port] / len(tt_df)

def port_to_prob_train_Dst(port : int) -> float:
    return dst_tt[port] / len(tt_df)

def port_to_prob_valid_Src(port : int) -> float:
    return src_vv[port] / len(vv_df)

def port_to_prob_valid_Dst(port : int) -> float:
    return dst_vv[port] / len(vv_df)

In [ ]:
tt_df['Src Port Ratio'] = tt_df['Src Port'].map(port_to_prob_train_Src)
vv_df['Src Port Ratio'] = vv_df['Src Port'].map(port_to_prob_valid_Src)
tt_df['Dst Port Ratio'] = tt_df['Dst Port'].map(port_to_prob_train_Dst)
vv_df['Dst Port Ratio'] = vv_df['Dst Port'].map(port_to_prob_valid_Dst)

In [ ]:
tt_df.head(3)

## Pearson Corr

In [ ]:
#for col, corr in tt_df.corr(method='pearson')['Label'].sort_values(0).items():
#    print(f'{corr} \t {col}')

# 모델 정의 학습

In [ ]:
import torch

import torch.nn as nn
import numpy as np

from tqdm.notebook import tqdm
from torch.optim import Adam, lr_scheduler
from torch.utils.data import DataLoader, Dataset


device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

class FlowDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]
    
class BinaryClassifier(nn.Module):
    def __init__(self, num_feature):
        super(BinaryClassifier, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(num_feature, 2048),
            nn.LayerNorm(2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        output = self.layer(x)
        return output

def train(train_X, train_y):
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # torch.backends.mps.deterministic = True
    # torch.backends.mps.benchmark = False
    np.random.seed(42)

    model = BinaryClassifier(len(train_X[0]))
    criterion = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr = 2e-4)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.85)
    epoch = 60

    train_dataset = FlowDataset(train_X, train_y)
    train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True)

    model.to(device)
    with tqdm(range(epoch)) as pbar:
        for _ in pbar:
            epoch_loss = 0
            data_count = 0
            for data, label in train_dataloader:
                data = data.to(device)
                label = label.to(device)
                output = model(data)
                loss = criterion(output.squeeze(1), label)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                data_count += len(data)
                pbar.set_postfix(epoch=epoch, loss=f"{epoch_loss / data_count}")
            scheduler.step()
    return model

def test(test_X, test_y, model):
    test_dataset = FlowDataset(test_X, test_y)
    test_dataloader = DataLoader(test_dataset, batch_size= 512, shuffle=False)
    model.to(device)
    pred = []
    with torch.no_grad():
        for data, label in (test_dataloader):
            data = data.to(device)
            label = label.to(device)
            output = model(data)
            output = output.cpu().squeeze().tolist()
            pred.extend(output)
    return pred

def test2(test_X, model):
    return  model(test_X)

# 모델 학습

In [ ]:
# drop_columns = ["Src IP", "Dst IP", "Label", 'Timestamp', 'Flags', 'Protocol']
# drop_columns = ["Src IP", "Dst IP", "Label", 'Timestamp', 'Protocol']
# drop_columns = ["Src IP", "Dst IP", "Label", 'Timestamp', 'Flags']
# drop_columns = ["Src IP", "Dst IP", "Label", 'Flags', 'Protocol']
# drop_columns = ["Src IP", "Dst IP", "Label", 'Timestamp']
# drop_columns = ["Src IP", "Dst IP", "Label", 'Protocol']
# drop_columns = ["Src IP", "Dst IP", "Label", 'Flags']
drop_columns = ["Src IP", "Dst IP", "Label"]

In [ ]:
tt_list = tt_df.drop(drop_columns, axis=1).values.tolist()
vv_list = vv_df.drop(drop_columns, axis=1).values.tolist()

tt_y = tt_df['Label'].values.tolist()
vv_y = vv_df['Label'].values.tolist()

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# scaler = MinMaxScaler()
scaler = StandardScaler()
scaler.fit(tt_list)

StandardScaler()

In [ ]:
tt_X = scaler.transform(tt_list).astype(np.float32)
vv_X = scaler.transform(vv_list).astype(np.float32)

tt_y = np.array(tt_y, dtype=np.float32)
vv_y = np.array(vv_y, dtype=np.float32)

In [ ]:
# train 데이터 학습 및 valid 데이터 예측
model = train(tt_X, tt_y)

  0%|          | 0/60 [00:00<?, ?it/s]

# 모델 추론

In [ ]:
predict = test(vv_X, vv_y, model)
predict = np.array(predict, dtype="float32")

In [ ]:
# 성능 평가를 위해 예측 값을 반올림
round_valid_y = vv_y.astype("int")
round_predict = np.round(predict)

In [ ]:
# 플로우 기준 이상 탐지 성능 평가
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("confusion_matrix\n", confusion_matrix(round_valid_y, round_predict))
print(f"accuracy\t: {accuracy_score(round_valid_y, round_predict): .5f}")
print(f"precision\t: {precision_score(round_valid_y, round_predict): .5f}")
print(f"recall\t\t: {recall_score(round_valid_y, round_predict): .5f}")
print(f"F-1\t\t: {f1_score(round_valid_y, round_predict): .5f}")

# 성능 평가

In [ ]:
import csv
def CSVFileSave(total_ip: list):
    header = ["IP", "Prediction"]
    with open('IP_Predict.csv', 'w') as csvfile:
        writer = csv.writer(
            csvfile, dialect='excel', lineterminator='\n')
        writer.writerow(header)
        for data in total_ip:
            writer.writerow((data[0], data[1]))

In [ ]:
def get_outer_ip_dict(_theta: float):
    _outer_ip_dict = {}
    for _i in range(len(predict)):
        oip = valid_df.at[_i, 'Src IP']
        if oip not in labelIP_dict:
            oip = valid_df.at[_i, 'Dst IP']

        if predict[_i] >= _theta:
            _outer_ip_dict[oip] = _outer_ip_dict.get(oip, np.array([0, 0])) + (1, 1)
        else:
            _outer_ip_dict[oip] = _outer_ip_dict.get(oip, np.array([0, 0])) + (0, 1)

    return _outer_ip_dict

def get_pred_ip(_outer_ip_dict: dict, _threshold: float):
    _pred_ip_list = []
    total_IP = []
    for oip, (numerator, denominator) in _outer_ip_dict.items():
        total_IP.append([oip, int(numerator / denominator >= _threshold)])
        _pred_ip_list.append(int(numerator / denominator >= _threshold))
    CSVFileSave(total_IP)
    return _pred_ip_list

def get_label_ip(_outer_ip_dict: dict):
    _label_ip = []

    for oip in _outer_ip_dict.keys():
        _label_ip.append(labelIP_dict[oip])

    return _label_ip

In [ ]:
_theta = 0.1
_threshold = 0.1

_outer_ip_dict = get_outer_ip_dict(_theta)
_pred_ip = get_pred_ip(_outer_ip_dict, _threshold)

In [ ]:
def get_f1_score(_theta, _threshold):
    _outer_ip_dict = get_outer_ip_dict(_theta)
    _label_ip = get_label_ip(_outer_ip_dict)
    _pred_ip = get_pred_ip(_outer_ip_dict, _threshold)
    return f1_score(_label_ip, _pred_ip)

In [ ]:
def pred_test(_theta: float, _threshold: float):
    _outer_ip_dict = get_outer_ip_dict(_theta)
    label_IP = get_label_ip(_outer_ip_dict)
    pred_IP = get_pred_ip(_outer_ip_dict, _threshold)

    print("confusion_matrix\n", confusion_matrix(label_IP, pred_IP))
    print(f"accuracy\t: {accuracy_score(label_IP, pred_IP): .5f}")
    print(f"precision\t: {precision_score(label_IP, pred_IP): .5f}")
    print(f"recall\t\t: {recall_score(label_IP, pred_IP): .5f}")
    print(f"F-1\t\t: {f1_score(label_IP, pred_IP): .5f}")

In [ ]:
# 정답 IP dictionary와 예측 IP dictionary를 비교하여 성능 평가
theta = 0.4
threshold = 0.55

pred_test(theta, threshold)

confusion_matrix
 [[576  14]
 [  5 452]]
accuracy	:  0.98185
precision	:  0.96996
recall		:  0.98906
F-1		:  0.97941


In [ ]:
from itertools import cycle

def max_threshold(_theta: float):
    _prev = [.05, .95]
    _res = Counter()
    pbar = tqdm(range(10))
    for _, cnt in zip(pbar, cycle([6, 5])):
        for _threshold in np.linspace(min(_prev), max(_prev), cnt):
            _threshold = np.round(_threshold, 2)
            if _threshold not in _res:
                pbar.set_description(f"Theta {_theta} - {_res.most_common(1)}")
                _res[_threshold] = get_f1_score(_theta, _threshold)
        _prev = list(map(lambda x: x[0], _res.most_common(3)))
    return _res.most_common(1)[0]

def max_theta():
    _prev = [.05, .95]
    _res = {}
    pbar = tqdm(range(10))
    for _, cnt in zip(pbar, cycle([6, 5])):
        for _theta in np.linspace(min(_prev), max(_prev), cnt):
            _theta = np.round(_theta, 2)
            if _theta not in _res:
                pbar.set_description(f"Current Theta - {_theta}")
                _res[_theta] = max_threshold(_theta)
        _prev = sorted(_res.keys(), key=lambda k: -_res[k][1])[:3]
    return sorted(_res.items(), key=lambda e: -e[1][1])

In [ ]:
max_threshold(0.4)

  0%|          | 0/10 [00:00<?, ?it/s]

(0.54, 0.9794149512459371)

In [ ]:
res = max_theta()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
pp(res)

[(0.46, (0.54, 0.9815418023887079)),
 (0.48, (0.59, 0.9814612868047982)),
 (0.47, (0.59, 0.9814612868047982)),
 (0.45, (0.54, 0.9804772234273319)),
 (0.5, (0.59, 0.9803493449781658)),
 (0.49, (0.59, 0.9803493449781658)),
 (0.41, (0.54, 0.9794149512459371)),
 (0.43, (0.54, 0.9794149512459371)),
 (0.55, (0.52, 0.9793253536452666)),
 (0.77, (0.43, 0.9782608695652174)),
 (0.59, (0.52, 0.9782135076252724)),
 (0.63, (0.52, 0.9782135076252724)),
 (0.23, (0.59, 0.9772481040086674)),
 (0.68, (0.45, 0.9751887810140237)),
 (0.7, (0.53, 0.9746974697469748)),
 (0.05, (0.59, 0.9722814498933903)),
 (0.95, (0.15, 0.9698324022346368))]


In [ ]:
pred_test(res[0][0], res[0][1][0])

confusion_matrix
 [[578  12]
 [  5 452]]
accuracy	:  0.98376
precision	:  0.97414
recall		:  0.98906
F-1		:  0.98154
